In [1]:
%load_ext autoreload
%autoreload 2 

from datasets import load_dataset
import os
import matplotlib.pyplot as plt
import sys
from PIL import Image
import cv2
import numpy as np

from src.image_preprocessor_utils import ImageUtils

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from common.common_methods import CommonMethods

/home/bsuetcueoglu/testprojectthesis/thesistest/thesisProjectTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Resize images for Stable Diffusion and Resize back the Generated Images
- Loads dataset to HuggingFace

Select resizing alg. lanczos, bicubic or letterbox and move the prompts (metadata) file to resized images folder

In [2]:
config_file_path = "./preprocessor_config.yaml"
config = CommonMethods.read_yaml(config_file_path)

input_images_dir = config["dataloader"]["images_to_be_resized_path"]
resized_images_dir = config["dataloader"]["resized_images_dir"]

In [ ]:
ImageUtils.resize_all_images(input_images_dir, resized_images_dir, "lanczos", 1024, 1024)
ImageUtils.copy_metadata_file_to_resized_images_dir(input_images_dir, resized_images_dir) # metadata includes labels of the images

86 images are resized to 1024x1024 using lanczos and saved to ../Data/resized_fba_with_lanczos_wo_wearscores_refactoredlabels
Metadata file metadata.jsonl copied to ../Data/resized_fba_with_lanczos_wo_wearscores_refactoredlabels


Create HuggingFace dataset
- This dataset is going to be used for finetuning Stable Diffusion.

In [15]:
dataset = load_dataset("imagefolder", data_dir=resized_images_dir, drop_labels="True")

huggingface_dataset_link = config["dataloader"]["huggingface_dataset_link"]
dataset.push_to_hub(huggingface_dataset_link)

Generating train split: 86 examples [00:00, 14582.40 examples/s]
Uploading the dataset shards: 100%|██████████| 1/1 [00:27<00:00, 27.47s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/artisanalwasp/resized_fba_with_lanczos_wo_wearscores_refactoredlabels/commit/b203907d297bf3d38faacc1094974e71816048ad', commit_message='Upload dataset', commit_description='', oid='b203907d297bf3d38faacc1094974e71816048ad', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/artisanalwasp/resized_fba_with_lanczos_wo_wearscores_refactoredlabels', endpoint='https://huggingface.co', repo_type='dataset', repo_id='artisanalwasp/resized_fba_with_lanczos_wo_wearscores_refactoredlabels'), pr_revision=None, pr_num=None)

In [16]:
hf_dataset = load_dataset(huggingface_dataset_link)
print(hf_dataset)

Generating train split: 100%|██████████| 86/86 [00:01<00:00, 79.33 examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 86
    })
})


--------------------------------------------------------------------------------------------------------------------------------------------------

Resize the generated images to the original image size to be used in training Stable Diffusion Model

In [ ]:
%load_ext autoreload
%autoreload 2 

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from common.common_methods import CommonMethods
from src.image_preprocessor_utils import ImageUtils

config_file_path = "./preprocessor_config.yaml"
config = CommonMethods.read_yaml(config_file_path)

generated_im_dir = config["dataloader"]["generated_images_dir"]
resized_gen_im_dir = "../Data/FBA_dataset/generated/bue/gen15/resized"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
ImageUtils.resize_all_images(generated_im_dir, resized_gen_im_dir, "lanczos", target_image_width=1280, target_image_height=1024)
ImageUtils.png_to_bmp_all_images(resized_gen_im_dir, "../Data/FBA_dataset/generated/bue/gen15/resized_bmp")